In [1]:
%load_ext line_profiler

In [2]:
import numpy as np
from sage.all import *
from mubs_utils import testMubs, saveMubs
reset()

In [3]:
p = 2
n = 2
q = p**n

# GR(4,2)
R  = PolynomialRing(Integers(4), 't'); t = R.gen()
GR = R.quotient(t**2+t+1, 'w'); w = GR.gen()
GR

Univariate Quotient Polynomial Ring in w over Ring of integers modulo 4 with modulus t^2 + t + 1

In [4]:
GR.cardinality()

16

In [5]:
list(GR)

[0,
 1,
 2,
 3,
 w,
 w + 1,
 w + 2,
 w + 3,
 2*w,
 2*w + 1,
 2*w + 2,
 2*w + 3,
 3*w,
 3*w + 1,
 3*w + 2,
 3*w + 3]

In [6]:
# def generalizedFrobenius(a, b):
#     return a**2 + GR(2)*b**2

def ringTrace(a, b):
    s = GR(0)
    for k in range(n):
        s += a**(2**k) + GR(2)*b**(2**k)
    return s 

for a in GR:
    for b in GR:
        if not ringTrace(a, b) == a + GR(2)*b + a**2 + GR(2)*b**2:
            raise Error('Trace not well defined!')
print('Trace defined!')

Trace defined!


In this case the ring generator $w$ coincides with the $\xi$.

In [7]:
w**3

1

In [8]:
T = [0] + [w**k for k in range((2**n - 2) + 1)]
T

[0, 1, w, 3*w + 3]

In [9]:
def brute2adic(x):
    for a in T:
        for b in T:
            if x == a + GR(2)*b:
                return (a,b)

In [10]:
mubs = [identity_matrix(q)]
for m in T:
    Bm = zero_matrix(SR, q, q)
    for j, v in enumerate(T):
        for i, w in enumerate(T):
            a, b = brute2adic(w**2 * m + 2 * w * v)
            pwr  = Integer(lift(ringTrace(a, b)))
            Bm[i,j] = 1/sqrt(q) * I**pwr
    mubs.append(Bm)

In [11]:
for i, m in enumerate(T):
    print(m)
    print(mubs[i+1])
    print('\n')

0
[ 1/2  1/2  1/2  1/2]
[ 1/2  1/2 -1/2 -1/2]
[ 1/2 -1/2 -1/2  1/2]
[ 1/2 -1/2  1/2 -1/2]


1
[   1/2    1/2    1/2    1/2]
[  -1/2   -1/2    1/2    1/2]
[-1/2*I  1/2*I  1/2*I -1/2*I]
[-1/2*I  1/2*I -1/2*I  1/2*I]


w
[   1/2    1/2    1/2    1/2]
[-1/2*I -1/2*I  1/2*I  1/2*I]
[  -1/2    1/2    1/2   -1/2]
[-1/2*I  1/2*I -1/2*I  1/2*I]


3*w + 3
[   1/2    1/2    1/2    1/2]
[-1/2*I -1/2*I  1/2*I  1/2*I]
[-1/2*I  1/2*I  1/2*I -1/2*I]
[  -1/2    1/2   -1/2    1/2]




In [12]:
from mubs_utils import saveMubs
saveMubs(mubs, 'MUBS/2-2-desarguesian')

Saved to MUBS/2-2-desarguesian


In [13]:
def testMubs(mubs):
    for B in mubs:
        for V in mubs:
            if B != V:
                for j in range(4):
                    for k in range(4):
                        r = (transpose(B[:,j]) * V[:,k]).norm().abs()**2
                        if r != 1/4:
                            print(r)
                            raise Exception('Not MUBs!')
    return True

In [14]:
testMubs(mubs)

True

Now for five qubits, $\mathcal H = \mathbb C^{2^5}$.

In [15]:
reset()

p = 2
n = 5
q = p**n

In [16]:
F = GF(2**5, 'w', repr='poly'); w = F.gen()
F

Finite Field in w of size 2^5

In [17]:
charpoly(w) # defaults to Conway polynomial

x^5 + x^2 + 1

In [18]:
# GR(4,5)
R  = PolynomialRing(Integers(4), 't'); t = R.gen()

In [19]:
factor(t**5 + t**2 + 1)

t^5 + t^2 + 1

In [20]:
GR = R.quotient(t**5 + t**2 + 1, 'alpha'); alpha = GR.gen()
GR

Univariate Quotient Polynomial Ring in alpha over Ring of integers modulo 4 with modulus t^5 + t^2 + 1

In [21]:
alpha**5 + alpha**2 + 1

0

Ring generator $\alpha$ is not the multiplicative generator of the Teichmüller set as can be seen below:

In [22]:
alpha**(2**5-1)

2*alpha^3 + 2*alpha + 1

Andrés found the $2^5-1$-th primitive root of unity though:

In [23]:
xi = alpha**2 + 2*alpha + 1
xi**(2**5-1)

1

In [24]:
T = [0] + [xi**k for k in range(2**n - 2 + 1)]
len(T)

32

In [25]:
T[-1]*xi

1

In [26]:
T # coincides with Andre's MAGMA code.

[0,
 1,
 alpha^2 + 2*alpha + 1,
 alpha^4 + 2*alpha^2 + 1,
 3*alpha^4 + 3*alpha^3 + alpha^2 + alpha + 3,
 2*alpha^4 + 3*alpha^3 + alpha^2 + 2,
 alpha^4 + 3*alpha^3 + 2*alpha + 3,
 3*alpha^4 + 2*alpha^2 + 3*alpha + 2,
 alpha^4,
 alpha^4 + 3*alpha^3 + 2*alpha^2 + 3*alpha + 2,
 alpha^4 + alpha^3 + alpha^2 + 2*alpha + 1,
 3*alpha^2 + 3*alpha + 2,
 3*alpha^4 + alpha^3 + 3*alpha^2 + 3*alpha + 2,
 3*alpha^3 + 3,
 2*alpha^4 + 3*alpha^3 + 2*alpha,
 3*alpha^3 + alpha^2 + 1,
 3*alpha^4 + alpha^3 + 3*alpha^2 + 2*alpha + 2,
 2*alpha^3 + 2*alpha^2 + 3*alpha + 3,
 2*alpha^4 + alpha^3 + alpha^2 + alpha + 1,
 alpha^4 + 2*alpha^3 + 3*alpha^2 + alpha,
 alpha^2,
 alpha^4 + 2*alpha^3 + alpha^2,
 2*alpha^4 + 3*alpha^3 + alpha^2 + 3*alpha,
 alpha^4 + 2*alpha^3 + alpha + 1,
 alpha^4 + 2*alpha^3 + 3*alpha^2 + 2*alpha + 1,
 alpha^3 + 3*alpha + 1,
 2*alpha^4 + 2*alpha^2 + alpha,
 3*alpha^3 + 3*alpha,
 2*alpha^4 + 2*alpha^3 + 3*alpha^2 + 3*alpha + 1,
 alpha^4 + alpha^3 + 3*alpha + 3,
 3*alpha^4 + 3*alpha^3 + 2*alp

In [27]:
xi

alpha^2 + 2*alpha + 1

In [28]:
def ringTrace(a, b):
    s = GR(0)
    for k in range(n):
        s += a**(2**k) + GR(2)*b**(2**k)
    return s 

In [29]:
def brute2adic(x):
    for a in T:
        for b in T:
            if x == a + GR(2)*b:
                return (a,b)

In [30]:
def dumb2adic(x):
    for b in T:
        a = x - GR(2)*b
        if a in T:
            return (a,b)

Kantor mubs.

In [31]:
def createMubs(k=None):
    mubs = [identity_matrix(q)]
    for m in T[:k]:
        Bm = zero_matrix(SR, q, q)
        for j, v in enumerate(T):
            for i, w in enumerate(T):
                wom = m**2*w + m*ringTrace(w, 0) + ringTrace(m*w, 0)
                a, b = dumb2adic(w * wom + GR(2) * w * v)
                pwr  = Integer(lift(ringTrace(a, b)))
                Bm[i,j] = 1/sqrt(q) * I**pwr
        mubs.append(Bm)
    return mubs

In [32]:
%lprun -f createMubs createMubs(k=2)

Timer unit: 1e-09 s

Total time: 6.68206 s
File: /var/folders/33/k06yq9x95jqcnbk_bh41j4qc0000gn/T/ipykernel_30444/2185357466.py
Function: createMubs at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def createMubs(k=None):
     2         1     537000.0 537000.0      0.0      mubs = [identity_matrix(q)]
     3         2       3000.0   1500.0      0.0      for m in T[:k]:
     4         2     513000.0 256500.0      0.0          Bm = zero_matrix(SR, q, q)
     5        64      81000.0   1265.6      0.0          for j, v in enumerate(T):
     6      2048    3708000.0   1810.5      0.1              for i, w in enumerate(T):
     7      2048 2122053000.0 1036158.7     31.8                  wom = m**2*w + m*ringTrace(w, 0) + ringTrace(m*w, 0)
     8      2048 2879810000.0 1406157.2     43.1                  a, b = dumb2adic(w * wom + GR(2) * w * v)
     9      2048 1183679000.0 577968.3     17.7                  pwr  = I

In [33]:
mubs = createMubs()

In [36]:
saveMubs(mubs, 'MUBS/2-5-kantor')

Saved to MUBS/2-5-kantor


In [39]:
M = np.load('MUBS/2-5-kantor.npy')
M.shape

(1056, 32)

Desarguesian spread.

In [40]:
def createMubs(k=None):
    mubs = [identity_matrix(q)]
    for m in T[:k]:
        Bm = zero_matrix(SR, q, q)
        for j, v in enumerate(T):
            for i, w in enumerate(T):
                a, b = dumb2adic(w**2 * m + 2 * w * v)
                pwr  = Integer(lift(ringTrace(a, b)))
                Bm[i,j] = 1/sqrt(q) * I**pwr
        mubs.append(Bm)
    return mubs

In [41]:
mubs = createMubs()

In [45]:
saveMubs(mubs, 'MUBS/2-5-desarguesian')

Saved to MUBS/2-5-desarguesian
